<a href="https://colab.research.google.com/github/arintaauza/Qiskit/blob/main/QFTArith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quantum Addition and Multiplication

In [ ]:
!pip install qiskit==0.16.1

Consider a $d$-dimensional system with state $|x\rangle$ from the computation basis $\{|0\rangle, |1\rangle, \ldots, |d-1\rangle\}$. The Quantum Fourier Transform of $|x\rangle$ is 


\begin{equation}
   |\phi(x)\rangle= QFT|x\rangle = \frac{1}{\sqrt{d}} \sum_{k=0}^{d-1} e^{i \frac{2 \pi xk}{d}}|k\rangle
\end{equation}


We define $CZ_{r,s}$ as the controlled-$Z$ gate with the $r$-th qubit as the control qubit and the $s$-th qubit as the target qubit. We can also define
\begin{equation*}
    CZ^F |x\rangle|y\rangle = e^{i \frac{2 \pi xy}{Fd}} |x\rangle|y\rangle
\end{equation*}


#QFT ADDER

Let $x,y,d$ be integers. We want to compute the sum $x+y (\text{mod } d)$. In order to compute the addition ($\text{mod }d$), we do the following operation.
\begin{equation} 
    IQFT_2 \cdot CZ \cdot QFT_2 |x\rangle |y\rangle = |x\rangle |x + y (\text{mod } d)\rangle
\end{equation}


In [ ]:
# -*- coding: utf-8 -*-

"""
Arinta Auza - University of Technology Sydney
Rakesh Saini - Macquarie University
"""

import math
import operator
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit
from qiskit import execute
import matplotlib as mpl
from qiskit import *

def createInputState(qc, reg, n, pie):
    """
    Computes the quantum Fourier transform of reg, one qubit at
    a time.
    Apply one Hadamard gate to the nth qubit of the quantum register reg, and 
    then apply repeated phase rotations with parameters being pi divided by 
    increasing powers of two.
    """ 
    qc.h(reg[n])    
    for i in range(0, n):
        qc.cu1(pie/float(2**(i+1)), reg[n-(i+1)], reg[n])    
    
def evolveQFTState(qc, reg_a, reg_b, n, pie):
    """
    
    Evolves the state |F(ψ(reg_a))> to |F(ψ(reg_a+reg_b))> using the quantum 
    Fourier transform conditioned on the qubits of the reg_b.
    Apply repeated phase rotations with parameters being pi divided by 
    increasing powers of two.
    """
    for i in range(0, n+1):
        qc.cu1(pie/float(2**(i)), reg_b[n-i], reg_a[n])

def evolveQFTState2(qc, reg_a, reg_b, n, pie, factor):
    """  
    Evolves the state |F(ψ(reg_a))> to |F(ψ(reg_a+reg_b))> using the quantum 
    Fourier transform conditioned on the qubits of the reg_b.
    Apply repeated phase rotations with parameters being pi divided by 
    increasing powers of two.
    """
    l = len(reg_b)
    for i in range(0, n + 1):
        if (n - i) > l - 1:
            pass
        else:
            qc.cu1(factor*pie / float(2**(i)), reg_b[n - i], reg_a[n])

def inverseQFT(qc, reg, n, pie):
    """
    Performs the inverse quantum Fourier transform on a register reg.
    Apply repeated phase rotations with parameters being pi divided by 
    decreasing powers of two, and then apply a Hadamard gate to the nth qubit
    of the register reg.
    """
    for i in range(0, n):
        qc.cu1(-1*pie/float(2**(n-i)), reg[i], reg[n])
    qc.h(reg[n])

def add(first, second, n):
    pie = math.pi

    a = QuantumRegister(n+1, "a") 
    b = QuantumRegister(n+1, "b")     
    cl = ClassicalRegister(n+1, "cl") 
    qc = QuantumCircuit(a, b, cl, name="qc")
    #Flip the corresponding qubit in register a if a bit in the string
    #first is a 1
    for i in range(0, n):
        if first[i] == "1":
            qc.x(a[n-(i+1)])
    #Flip the corresponding qubit in register b if a bit in the string
    #second is a 1
    for i in range(0, n):
        if second[i] == "1":
            qc.x(b[n-(i+1)])
    #Compute the Fourier transform of register a
    for i in range(0, n+1):
        createInputState(qc, a, n-i, pie)
    #Add the two numbers by evolving the Fourier transform F(ψ(reg_a))>
    #to |F(ψ(reg_a+reg_b))>
    for i in range(0, n+1):
        evolveQFTState(qc, a, b, n-i, pie) 
    #Compute the inverse Fourier transform of register a
    for i in range(0, n+1):
        inverseQFT(qc, a, i, pie)  
    qc.draw()
    #Measure qubits
    for i in range(0, n+1):
        qc.measure(a[i], cl[i])
    # Get measurement results of 1000 simulations

    if (n < 3):
      print(qc)

    backend = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend, shots = 1000)
    sim_result = job_sim.result()
    counts = sim_result.get_counts(qc)
    output = max(counts.items(), key=operator.itemgetter(1))[0]
    print(output)
    print("Addition:",int(output,2))
    # Display histogram of output
    visualization.plot_histogram(counts)

def add2(reg_a, reg_b, circ, factor):
    """
    Add two quantum registers reg_a and reg_b, and store the result in 
    reg_a.
    """
    pie = math.pi
    n = len(reg_a) - 1

    # Compute the Fourier transform of register a
    for i in range(0, n + 1):
        createInputState(circ, reg_a, n - i, pie)
    # Add the two numbers by evolving the Fourier transform F(ψ(reg_a))>
    # to |F(ψ(reg_a+reg_b))>
    for i in range(0, n + 1):
        evolveQFTState2(circ, reg_a, reg_b, n - i, pie, factor)
    # Compute the inverse Fourier transform of register a
    for i in range(0, n + 1):
        inverseQFT(circ, reg_a, i, pie)



#QFT Multiplier

Add the multiplicand to itself multiple times until we get the result. 

In [ ]:
def mult(first, second, n):
  multiplicand = QuantumRegister(n)
  multiplier = QuantumRegister(n)
  accumulator = QuantumRegister(2*n)
  cl = ClassicalRegister(2 *n)
  d = QuantumRegister(1)

  circ = QuantumCircuit(accumulator, multiplier, multiplicand, d, cl, name="qc")

  circ.x(d)
  # Store bit strings in quantum registers
  for i in range(n):
    if first[i] == '1':
        circ.x(multiplicand[l1 - i - 1])

  for i in range(n):
    if second[i] == '1':
        circ.x(multiplier[l1 - i - 1])

  multiplier_str = '1'
  # Perform repeated addition until the multiplier is zero
  while(int(multiplier_str) != 0):
    add2(accumulator, multiplicand, circ, 1)
    add2(multiplier, d, circ, -1)
    for i in range(len(multiplier)):
        circ.measure(multiplier[i], cl[i])
    result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
                    shots=2).result().get_counts(circ.name)
    multiplier_str = list(result.keys())[0]

  circ.measure(accumulator, cl)

  if (n < 3):
    print(circ)
  result = execute(circ, backend=Aer.get_backend('qasm_simulator'),
            shots=2).result().get_counts(circ.name)
  output = max(result.items(), key=operator.itemgetter(1))[0]
  

  print("Multiplication:", int(output,2))

In [ ]:
#Take two numbers as user input in binary form   
first = int(input("Enter a number with less than 10 digits."))
first = format(first,"b")
print(first)
l1 = len(first)
second = int(input("Enter another number with less than 10 digits."))
second = format(second,"b")
print(second)
l2 = len(second)
#Making sure that 'first' and 'second' are of the same length 
#by padding the smaller string with zeros
if l2>l1:
    first,second = second, first
    l2, l1 = l1, l2
second = ("0")*(l1-l2) + second
print("l1=",l1)
add(first, second, l1)
mult(first, second, l1)
  

Enter a number with less than 10 digits.2
10
Enter another number with less than 10 digits.1
1
l1= 2
                                            ┌───┐                  ┌───┐»
a_0: |0>─────────────■───────────■──────────┤ H ├──────────────■───┤ H ├»
        ┌───┐        │     ┌───┐ │pi/pi/2   └───┘              │   └───┘»
a_1: |0>┤ X ├─■──────┼─────┤ H ├─■─────────────────■────■──────┼────────»
        ├───┤ │pi/2  │pi/4 └───┘                   │    │      │        »
a_2: |0>┤ H ├─■──────■──────■──────■────────■──────┼────┼──────┼────────»
        ├───┤               │      │        │pi/4  │    │pi/2  │pi      »
b_0: |0>┤ X ├───────────────┼──────┼────────■──────┼────■──────■────────»
        └───┘               │      │pi/2           │pi                  »
b_1: |0>────────────────────┼──────■───────────────■────────────────────»
                            │pi                                         »
b_2: |0>────────────────────■───────────────────────────────────────────»
           